# Get a subset of a state (by filtering for a city)

## Download the 2017 metadata and show where we put it

Use irsx_index at the command line to retrieve the 2017 listing of all xml 990 efilings.

__Note that these are filings received *during* 2017, so check the tax_period__

Irsx_index is a helper command that is included with irsx, so you need to have installed it first. Try `pip install irsx` or see more [here](https://github.com/jsfenfen/990-xml-reader/#installation).

We use the `--verbose` flag so can watch it's progress

    $ irsx_index --verbose --year=2017
    Getting index file for year: 2017 
    remote=https://s3.amazonaws.com/irs-form-990/index_2017.csv 
    local=/Users/jfenton/github-whitelabel/envs/irs-cookbook/lib/python3.6/site-packages/irsx/CSV/index_2017.csv
    Beginning streaming download of https://s3.amazonaws.com/irs-form-990/index_2017.csv
    Total file size: 59.45 MB

In [7]:
## You don't need to run the command below if you've run irs_index at the command line
## To actually do this from within the notebook environment uncomment the below
## Note that we're using the %sx 'magic command' which captures the output as an array 
## Your mileage may vary depending on how jupyter plays with your operating system

# %sx irsx_index --verbose --year=2017

# Get all nonprofit organizations in your state

We grabbed a file for just the state of Oregon as eo_or.csv from here: 
https://www.irs.gov/charities-non-profits/exempt-organizations-business-master-file-extract-eo-bmf

Note that this method isn't great for historic organizations; the IRS purges organizations after they've become inactive for a period of time. Historic EO BMF files are available here: http://nccs-data.urban.org/data.php?ds=bmf 

In [8]:
# importing libraries we'll use.
import csv
import os
import pandas as pd

# This tells us where the csv files are located in the system
from irsx.settings import INDEX_DIRECTORY

In [9]:
oregon_np = pd.read_csv("eo_or.csv")

In [10]:
# look at the first few lines
#oregon_np.head()
## print the headers as an array
list(oregon_np)

['EIN',
 'NAME',
 'ICO',
 'STREET',
 'CITY',
 'STATE',
 'ZIP',
 'GROUP',
 'SUBSECTION',
 'AFFILIATION',
 'CLASSIFICATION',
 'RULING',
 'DEDUCTIBILITY',
 'FOUNDATION',
 'ACTIVITY',
 'ORGANIZATION',
 'STATUS',
 'TAX_PERIOD',
 'ASSET_CD',
 'INCOME_CD',
 'FILING_REQ_CD',
 'PF_FILING_REQ_CD',
 'ACCT_PD',
 'ASSET_AMT',
 'INCOME_AMT',
 'REVENUE_AMT',
 'NTEE_CD',
 'SORT_NAME']

In [11]:
# Ignore some columns for now
or_np_simplified = oregon_np.filter(items=['EIN', 'NAME', 'ICO', 'STREET', 'CITY', 'STATE', 'ZIP', 'INCOME_AMT', 'ASSET_AMT', 'TAX_PERIOD'])
print("total oregon orgs: %s" % len(or_np_simplified))

# This is a toy filter for a demo -- you'd want something more robust than a perfect text match
pdx_orgs = or_np_simplified.query('CITY == "PORTLAND"')
print("total Portland, OR orgs: %s" % len(pdx_orgs))

total oregon orgs: 23956
total Portland, OR orgs: 5517


In [12]:
# Show the top values by income 
pdx_orgs.sort_values(by=['INCOME_AMT'], ascending=[0]).head()


,EIN,NAME,ICO,STREET,CITY,STATE,ZIP,INCOME_AMT,ASSET_AMT,TAX_PERIOD
15786,930798039,KAISER FOUNDATION HEALTH PLAN OF THE NORTHWEST,% SVP CC AND CAO,2701 NW VAUGHN ST STE 490,PORTLAND,OR,97210-5358,4.467836e+09,1.530943e+09,201612.0
16674,930863097,PROVIDENCE HEALTH PLAN,% KRISTY GARNER,4400 NE HALSEY BLDG 2,PORTLAND,OR,97213-1545,2.705925e+09,7.391670e+08,201612.0
9233,550828701,PROVIDENCE HEALTH ASSURANCE,% SHELLY M HANDKINS,4400 NE HALSEY BLDG 2,PORTLAND,OR,97213-1545,1.405434e+09,3.439671e+08,201612.0
6405,455093195,HEALTH SHARE OF OREGON,% JANET MEYER,2121 SW BROADWAY STE 200,PORTLAND,OR,97201-3181,1.258152e+09,9.271795e+07,201612.0
17312,930933975,CAREOREGON INC,% TERESA KENNEDY LEARN CFO,315 SW 5TH AVE,PORTLAND,OR,97204-1703,9.668015e+08,4.179530e+08,201612.0


In [13]:

# this is from the index file we dowloaded at the start
INDEX_2017= os.path.join(INDEX_DIRECTORY, 'index_2017.csv')
np_2017 = pd.read_csv(INDEX_2017)


## Now save the list of possible filers who actually filed out to .csv

In [14]:
# Now find orgs that are in portland that filed in 2017. 
# This join requires that both fields be named EIN and be formatted the same

pdx_2017_efilers = pd.merge(np_2017,
                 pdx_orgs,
                 on='EIN')
print("Found a total of %s portland 2017 efilers" % len(pdx_2017_efilers))
pdx_2017_efilers.head()

# sort by income amt, asset amt
pdx_2017_efilers = pdx_2017_efilers.sort_values(by=['INCOME_AMT', 'ASSET_AMT'], ascending=[0,0])
# Lets write them back out to a file for reference.
pdx_2017_efilers.to_csv('pdxefilers.csv')

# These are the top few for reference
pdx_2017_efilers.head()

Found a total of 1874 portland 2017 efilers


,RETURN_ID,FILING_TYPE,EIN,TAX_PERIOD_x,SUB_DATE,TAXPAYER_NAME,RETURN_TYPE,DLN,OBJECT_ID,NAME,ICO,STREET,CITY,STATE,ZIP,INCOME_AMT,ASSET_AMT,TAX_PERIOD_y
527,14174417,EFILE,930798039,201512,2/28/2017 5:56:33 PM,KAISER FOUNDATION HEALTH PLAN OF THE NORTHWEST,990,93493314032386,201633149349303238,KAISER FOUNDATION HEALTH PLAN OF THE NORTHWEST,% SVP CC AND CAO,2701 NW VAUGHN ST STE 490,PORTLAND,OR,97210-5358,4.467836e+09,1.530943e+09,201612.0
738,14274310,EFILE,930863097,201512,4/11/2017 5:11:29 PM,PROVIDENCE HEALTH PLAN,990O,93493320152136,201633209349315213,PROVIDENCE HEALTH PLAN,% KRISTY GARNER,4400 NE HALSEY BLDG 2,PORTLAND,OR,97213-1545,2.705925e+09,7.391670e+08,201612.0
739,15041026,EFILE,930863097,201612,12/21/2017 10:25:57 AM,PROVIDENCE HEALTH PLAN,990O,93493306015757,201703069349301575,PROVIDENCE HEALTH PLAN,% KRISTY GARNER,4400 NE HALSEY BLDG 2,PORTLAND,OR,97213-1545,2.705925e+09,7.391670e+08,201612.0
566,14214618,EFILE,550828701,201512,3/22/2017 10:30:39 PM,PROVIDENCE HEALTH ASSURANCE,990O,93493320152056,201603209349315205,PROVIDENCE HEALTH ASSURANCE,% SHELLY M HANDKINS,4400 NE HALSEY BLDG 2,PORTLAND,OR,97213-1545,1.405434e+09,3.439671e+08,201612.0
567,15007123,EFILE,550828701,201612,12/5/2017 5:30:38 PM,PROVIDENCE HEALTH ASSURANCE,990O,93493306015727,201723069349301572,PROVIDENCE HEALTH ASSURANCE,% SHELLY M HANDKINS,4400 NE HALSEY BLDG 2,PORTLAND,OR,97213-1545,1.405434e+09,3.439671e+08,201612.0
